In [25]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F

from torch.cuda.amp import GradScaler, autocast
from torch.utils.checkpoint import checkpoint

In [26]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
img_size = 224  # Input image size
patch_size = 16  # Size of image patches
embed_dim = 768  # Embedding dimension
depth = 12  # Number of transformer encoder layers
num_heads = 12  # Number of attention heads
mlp_ratio = 4.0  # Ratio of MLP hidden dimension to embedding dimension
qkv_bias = True  # Add bias to the query, key, and value projections
dropout_rate = 0.1  # Dropout rate
num_classes = 2  # Number of classes (binary classification)
batch_size = 32  # Batch size
num_epochs = 20  # Number of training epochs
learning_rate = 0.001  # Learning rate


In [27]:
# Data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Set the paths to your training and validation directories
train_dir = '/kaggle/input/morph-splitted/train'
val_dir = '/kaggle/input/morph-splitted/val'
test_dir = '/kaggle/input/morph-splitted/test'

# Load your dataset
train_set = datasets.ImageFolder(root=train_dir, transform=transform)
val_set = datasets.ImageFolder(root=val_dir, transform=transform)
test_set = datasets.ImageFolder(root=test_dir, transform=transform)

# Create data loaders
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)


In [28]:
# ViT implementation
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels=3, patch_size=16, embed_dim=768):
        super().__init__()
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)  # (batch_size, embed_dim, h//patch_size, w//patch_size)
        x = x.flatten(2).transpose(1, 2)  # (batch_size, n_patches, embed_dim)
        return x

class ViT(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, num_classes=1000, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4.0, qkv_bias=True, dropout_rate=0.1):
        super().__init__()
        self.patch_embed = PatchEmbedding(in_channels, patch_size, embed_dim)
        num_patches = (img_size // patch_size) ** 2
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(dropout_rate)
        self.transformer = Transformer(embed_dim, depth, num_heads, mlp_ratio, qkv_bias, dropout_rate)
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)
    def forward(self, x):
        x = self.patch_embed(x)  # (batch_size, n_patches, embed_dim)
        cls_tokens = self.cls_token.expand(x.shape[0], -1, -1)  # (batch_size, 1, embed_dim)
        x = torch.cat((cls_tokens, x), dim=1)  # (batch_size, n_patches+1, embed_dim)
        x = x + self.pos_embed  # (batch_size, n_patches+1, embed_dim)
        x = self.pos_drop(x)
        x = self.transformer(x)  # (batch_size, n_patches+1, embed_dim)
        x = self.norm(x)
        cls_final = x[:, 0]  # (batch_size, embed_dim)
        output = self.head(cls_final)  # (batch_size, num_classes)
        return output

class Transformer(nn.Module):
    def __init__(self, embed_dim, depth, num_heads, mlp_ratio=4.0, qkv_bias=True, dropout_rate=0.1):
        super().__init__()
        self.layers = nn.ModuleList([
            TransformerLayer(embed_dim, num_heads, mlp_ratio, qkv_bias, dropout_rate)
            for _ in range(depth)
        ])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
class TransformerLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_ratio=4.0, qkv_bias=True, dropout_rate=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout_rate, bias=qkv_bias)
        self.norm2 = nn.LayerNorm(embed_dim)
        mlp_hidden_dim = int(embed_dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(mlp_hidden_dim, embed_dim),
            nn.Dropout(dropout_rate)
        )

    def forward(self, x):
        # Pass the same input to query, key, and value
        x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        x = x + self.mlp(self.norm2(x))
        return x# Initialize the model
    
model = ViT(img_size=img_size, patch_size=patch_size, in_channels=3, num_classes=num_classes,
            embed_dim=embed_dim, depth=depth, num_heads=num_heads, mlp_ratio=mlp_ratio,
            qkv_bias=qkv_bias, dropout_rate=dropout_rate).to(device)

# Loss function and optimizer

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [29]:
# # Assuming your target labels are integers representing class indices
# # Convert target labels to one-hot encoding
# one_hot_targets = F.one_hot(labels, num_classes)

# # Now one_hot_targets will have a size of [batch_size, num_classes]


In [30]:
# criterion = nn.BCEWithLogitsLoss()

# # Calculate loss
# loss = criterion(outputs, one_hot_targets)

# optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [31]:
num_epochs = 5  # Number of training epochs

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 20)

    for batch_idx, (inputs, labels) in enumerate(train_loader, start=1):
        inputs, labels = inputs.to(device), labels.to(device)

        # One-hot encode the target labels
        labels = F.one_hot(labels, num_classes=num_classes).float()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}/{len(train_loader)}, Loss: {running_loss / batch_idx:.4f}")

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    # Evaluation on validation set
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    print("Evaluating on validation set...")

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # One-hot encode the target labels
            labels = F.one_hot(labels, num_classes=num_classes).float()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.argmax(dim=1)).sum().item()

        val_accuracy = 100 * correct / total
        val_loss /= len(val_loader)
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")
        print("-" * 20)


Epoch 1/5
--------------------
Batch 100/750, Loss: 0.8349
Batch 200/750, Loss: 0.7544
Batch 300/750, Loss: 0.7274
Batch 400/750, Loss: 0.7139
Batch 500/750, Loss: 0.7038
Batch 600/750, Loss: 0.6979
Batch 700/750, Loss: 0.6950
Epoch 1/5, Loss: 0.6937
Evaluating on validation set...
Validation Loss: 0.6680, Validation Accuracy: 62.50%
--------------------
Epoch 2/5
--------------------
Batch 100/750, Loss: 0.6655
Batch 200/750, Loss: 0.6687
Batch 300/750, Loss: 0.6686
Batch 400/750, Loss: 0.6694
Batch 500/750, Loss: 0.6670
Batch 600/750, Loss: 0.6678
Batch 700/750, Loss: 0.6672
Epoch 2/5, Loss: 0.6679
Evaluating on validation set...
Validation Loss: 0.6636, Validation Accuracy: 62.50%
--------------------
Epoch 3/5
--------------------
Batch 100/750, Loss: 0.6630
Batch 200/750, Loss: 0.6673
Batch 300/750, Loss: 0.6706
Batch 400/750, Loss: 0.6702
Batch 500/750, Loss: 0.6683
Batch 600/750, Loss: 0.6670
Batch 700/750, Loss: 0.6666
Epoch 3/5, Loss: 0.6656
Evaluating on validation set...
Val

In [32]:
# Evaluation on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 62.50%


In [ ]:
import torch
from torch.utils.data import DataLoader

# Function to create data loaders
def create_data_loader(data_dir, transform, batch_size):
    dataset = datasets.ImageFolder(data_dir, transform)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    return data_loader

# Define the transforms for the datasets
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

batch_size = 32

# Create data loaders for each dataset
fm_loader = create_data_loader('/kaggle/input/mad-benchmark/FaceMorpher', transform, batch_size)
mg1_loader = create_data_loader('/kaggle/input/mad-benchmark/MIPGAN_I', transform, batch_size)
mg2_loader = create_data_loader('/kaggle/input/mad-benchmark/MIPGAN_II', transform, batch_size)
oc_loader = create_data_loader('/kaggle/input/mad-benchmark/OpenCV', transform, batch_size)
wm_loader = create_data_loader('/kaggle/input/mad-benchmark/Webmorph', transform, batch_size)


In [34]:
data_loaders = [fm_loader, mg1_loader, mg2_loader, oc_loader, wm_loader]
results = []
losses = []

# Evaluate the model on each dataset
criterion = nn.BCEWithLogitsLoss()
model.eval()


ViT(
  (patch_embed): PatchEmbedding(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.1, inplace=False)
  (transformer): Transformer(
    (layers): ModuleList(
      (0-11): 12 x TransformerLayer(
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.1, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.1, inplace=False)
        )
      )
    )
  )
  (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (head): Linear(in_features=768, out_features=2, bias=True)
)

In [39]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

# Assuming these variables are defined somewhere in your code
# model, criterion, device, num_classes, data_loaders, results, losses

for loader_idx, data_loader in enumerate(data_loaders):
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0
    dataset_name = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"][loader_idx]
    print(f"Evaluating dataset: {dataset_name}")

    for inputs, labels in tqdm(data_loader, desc=f"Processing {dataset_name}", leave=False):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # One-hot encode the target labels
        labels = F.one_hot(labels, num_classes=num_classes).float()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Calculate predictions
        preds = torch.sigmoid(outputs) > 0.5

        # Update running metrics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels).item()
        total_samples += inputs.size(0)

    # Calculate average loss and accuracy
    test_loss = running_loss / total_samples
    test_accuracy = running_corrects / total_samples

    results.append(test_accuracy)
    losses.append(test_loss)
    print(f"{dataset_name} - Loss: {test_loss:.4f} Acc: {test_accuracy:.4f}")

# Print the final results
print("\nFinal Results:")
names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
for name, accuracy in zip(names, results):
    print(f"{name}: {accuracy:.4f}")


Evaluating dataset: FaceMorpher


FaceMorpher - Loss: 0.9013 Acc: 0.3389
Evaluating dataset: MIPGAN_I


MIPGAN_I - Loss: 0.9013 Acc: 0.3389
Evaluating dataset: MIPGAN_II


MIPGAN_II - Loss: 0.9012 Acc: 0.3392
Evaluating dataset: OpenCV


OpenCV - Loss: 0.9001 Acc: 0.3434
Evaluating dataset: Webmorph


Webmorph - Loss: 0.8380 Acc: 0.5795

Final Results:
FaceMorpher: 0.3389
MIPGAN_I: 0.3389
MIPGAN_II: 0.3392
OpenCV: 0.3434
Webmorph: 0.5795


In [51]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from sklearn.metrics import roc_curve
from tqdm import tqdm

# Function to create data loaders
def create_data_loader(data_dir, transform, batch_size):
    dataset = datasets.ImageFolder(data_dir, transform)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    return data_loader

# Define the transforms for the datasets
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

batch_size = 32

# Create data loaders for each dataset
fm_loader = create_data_loader('/kaggle/input/mad-benchmark/FaceMorpher', transform, batch_size)
mg1_loader = create_data_loader('/kaggle/input/mad-benchmark/MIPGAN_I', transform, batch_size)
mg2_loader = create_data_loader('/kaggle/input/mad-benchmark/MIPGAN_II', transform, batch_size)
oc_loader = create_data_loader('/kaggle/input/mad-benchmark/OpenCV', transform, batch_size)
wm_loader = create_data_loader('/kaggle/input/mad-benchmark/Webmorph', transform, batch_size)
data_loaders = [fm_loader, mg1_loader, mg2_loader, oc_loader, wm_loader]

In [61]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from sklearn.metrics import roc_curve
from tqdm import tqdm

# Function to create data loaders
def create_data_loader(data_dir, transform, batch_size):
    dataset = datasets.ImageFolder(data_dir, transform)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    return data_loader

# Define the transforms for the datasets
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

batch_size = 32

# Create data loaders for each dataset
fm_loader = create_data_loader('/kaggle/input/mad-benchmark/FaceMorpher', transform, batch_size)
mg1_loader = create_data_loader('/kaggle/input/mad-benchmark/MIPGAN_I', transform, batch_size)
mg2_loader = create_data_loader('/kaggle/input/mad-benchmark/MIPGAN_II', transform, batch_size)
oc_loader = create_data_loader('/kaggle/input/mad-benchmark/OpenCV', transform, batch_size)
wm_loader = create_data_loader('/kaggle/input/mad-benchmark/Webmorph', transform, batch_size)
data_loaders = [fm_loader, mg1_loader, mg2_loader, oc_loader, wm_loader]

# Define fixed BPCER and APCER values for evaluation
fixed_bpcer_values = [0.01, 0.05, 0.1]
fixed_apcer_values = [0.01, 0.05, 0.1]

# Function to calculate metrics
def calculate_apcer(true_labels, predictions, fixed_bpcer):
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    fpr_target = fixed_bpcer
    closest_fpr_index = np.argmin(np.abs(fpr - fpr_target))
    apcer = 1 - tpr[closest_fpr_index]
    return apcer

def calculate_bpcer(true_labels, predictions, fixed_apcer):
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    tpr_target = 1 - fixed_apcer
    closest_tpr_index = np.argmin(np.abs(tpr - tpr_target))
    bpcer = fpr[closest_tpr_index]
    return bpcer

def calculate_eer(true_labels, predictions):
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    frr = 1 - tpr
    eer_index = np.argmin(np.abs(fpr - frr))
    eer = fpr[eer_index]
    return eer

# Evaluate the model on each dataset
criterion = torch.nn.BCEWithLogitsLoss()
model.eval()

results = []
losses = []
all_results = []

for loader_idx, data_loader in enumerate(data_loaders):
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0
    dataset_name = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"][loader_idx]
    print(f"Evaluating dataset: {dataset_name}")

    all_predictions = []
    all_true_labels = []

    for inputs, labels in tqdm(data_loader, desc=f"Processing {dataset_name}", leave=False):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # One-hot encode the target labels
        labels_one_hot = F.one_hot(labels, num_classes=num_classes).float()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels_one_hot)

        # Calculate predictions
        preds = torch.sigmoid(outputs) > 0.5

        # Update running metrics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels_one_hot.data).item()
        total_samples += inputs.size(0)

        # Store predictions and labels
        all_predictions.append(torch.sigmoid(outputs).detach().cpu().numpy())
        all_true_labels.append(labels_one_hot.cpu().numpy())

    # Calculate average loss and accuracy
    test_loss = running_loss / total_samples
    test_accuracy = running_corrects / total_samples

    results.append(test_accuracy)
    losses.append(test_loss)

    # Concatenate all predictions and true labels
    predictions = np.concatenate(all_predictions).ravel()
    true_labels = np.concatenate(all_true_labels).ravel()

    # Ensure the lengths of true labels and predictions match
    assert true_labels.shape[0] == predictions.shape[0], f"Lengths of true labels ({true_labels.shape[0]}) and predictions ({predictions.shape[0]}) do not match"

    # Calculate APCER, BPCER, and EER
    apcer_values = []
    bpcer_values = []

    for fixed_bpcer in fixed_bpcer_values:
        apcer = calculate_apcer(true_labels, predictions, fixed_bpcer)
        apcer_values.append(apcer)

    for fixed_apcer in fixed_apcer_values:
        bpcer = calculate_bpcer(true_labels, predictions, fixed_apcer)
        bpcer_values.append(bpcer)

    eer = calculate_eer(true_labels, predictions)

    # Store results
    result = {
        "Dataset": dataset_name,
        "Loss": test_loss,
        "Accuracy": test_accuracy,
        "APCER": apcer_values,
        "BPCER": bpcer_values,
        "EER": eer
    }
    all_results.append(result)

    # Print results
    print(f"{dataset_name} - Loss: {test_loss:.4f} Acc: {test_accuracy:.4f}")
    print(f"{dataset_name} - APCER: {apcer_values}, BPCER: {bpcer_values}, EER: {eer:.4f}")

# Print the final results
print("\nFinal Results:")
for result in all_results:
    print(result)

import pandas as pd

# Convert the final results to a DataFrame
df_results = pd.DataFrame(all_results)

# Print the final results
print("\nFinal Results:")
print(df_results)
df_results.head()

Evaluating dataset: FaceMorpher


FaceMorpher - Loss: 0.9013 Acc: 0.3389
FaceMorpher - APCER: [1.0, 1.0, 1.0], BPCER: [1.0, 1.0, 1.0], EER: 0.8306
Evaluating dataset: MIPGAN_I


MIPGAN_I - Loss: 0.9013 Acc: 0.3389
MIPGAN_I - APCER: [1.0, 1.0, 1.0], BPCER: [1.0, 1.0, 1.0], EER: 0.8306
Evaluating dataset: MIPGAN_II


MIPGAN_II - Loss: 0.9012 Acc: 0.3392
MIPGAN_II - APCER: [1.0, 1.0, 1.0], BPCER: [1.0, 1.0, 1.0], EER: 0.8304
Evaluating dataset: OpenCV


OpenCV - Loss: 0.9001 Acc: 0.3434
OpenCV - APCER: [1.0, 1.0, 1.0], BPCER: [1.0, 1.0, 1.0], EER: 0.8283
Evaluating dataset: Webmorph


Webmorph - Loss: 0.8380 Acc: 0.5795
Webmorph - APCER: [1.0, 1.0, 1.0], BPCER: [1.0, 1.0, 1.0], EER: 0.7102

Final Results:
{'Dataset': 'FaceMorpher', 'Loss': 0.9012874087621999, 'Accuracy': 0.3388704318936877, 'APCER': [1.0, 1.0, 1.0], 'BPCER': [1.0, 1.0, 1.0], 'EER': 0.8305647840531561}
{'Dataset': 'MIPGAN_I', 'Loss': 0.9012874087621999, 'Accuracy': 0.3388704318936877, 'APCER': [1.0, 1.0, 1.0], 'BPCER': [1.0, 1.0, 1.0], 'EER': 0.8305647840531561}
{'Dataset': 'MIPGAN_II', 'Loss': 0.9012133206611659, 'Accuracy': 0.33915211970074816, 'APCER': [1.0, 1.0, 1.0], 'BPCER': [1.0, 1.0, 1.0], 'EER': 0.830423940149626}
{'Dataset': 'OpenCV', 'Loss': 0.9000870469041946, 'Accuracy': 0.3434343434343434, 'APCER': [1.0, 1.0, 1.0], 'BPCER': [1.0, 1.0, 1.0], 'EER': 0.8282828282828283}
{'Dataset': 'Webmorph', 'Loss': 0.8379871276291934, 'Accuracy': 0.5795454545454546, 'APCER': [1.0, 1.0, 1.0], 'BPCER': [1.0, 1.0, 1.0], 'EER': 0.7102272727272727}



Final Results:
       Dataset      Loss  Accuracy            APCER            BPCER       EER
0  FaceMorpher  0.901287  0.338870  [1.0, 1.0, 1.0]  [1.0, 1.0, 1.0]  0.830565
1     MIPGAN_I  0.901287  0.338870  [1.0, 1.0, 1.0]  [1.0, 1.0, 1.0]  0.830565
2    MIPGAN_II  0.901213  0.339152  [1.0, 1.0, 1.0]  [1.0, 1.0, 1.0]  0.830424
3       OpenCV  0.900087  0.343434  [1.0, 1.0, 1.0]  [1.0, 1.0, 1.0]  0.828283
4     Webmorph  0.837987  0.579545  [1.0, 1.0, 1.0]  [1.0, 1.0, 1.0]  0.710227


,Dataset,Loss,Accuracy,APCER,BPCER,EER
0,FaceMorpher,0.901287,0.338870,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",0.830565
1,MIPGAN_I,0.901287,0.338870,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",0.830565
2,MIPGAN_II,0.901213,0.339152,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",0.830424
3,OpenCV,0.900087,0.343434,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",0.828283
4,Webmorph,0.837987,0.579545,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",0.710227


In [65]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm import tqdm

def calculate_apcer(true_labels, predictions, fixed_bpcer):
    """Calculate APCER at a fixed BPCER."""
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    fpr_target = fixed_bpcer
    closest_fpr_index = np.argmin(np.abs(fpr - fpr_target))
    apcer = 1 - tpr[closest_fpr_index]
    return apcer

def calculate_bpcer(true_labels, predictions, fixed_apcer):
    """Calculate BPCER at a fixed APCER."""
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    tpr_target = 1 - fixed_apcer
    closest_tpr_index = np.argmin(np.abs(tpr - tpr_target))
    bpcer = fpr[closest_tpr_index]
    return bpcer

def calculate_eer(true_labels, predictions):
    """Calculate EER."""
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    frr = 1 - tpr
    eer_index = np.argmin(np.abs(fpr - frr))
    eer = fpr[eer_index]
    return eer

# Assuming these variables are defined somewhere in your code
# model, criterion, device, num_classes, data_loaders, results, losses, fixed_bpcer_values, fixed_apcer_values

all_results = []

for loader_idx, data_loader in enumerate(data_loaders):
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0
    dataset_name = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"][loader_idx]
    print(f"Evaluating dataset: {dataset_name}")

    all_predictions = []
    all_true_labels = []

    for inputs, labels in tqdm(data_loader, desc=f"Processing {dataset_name}", leave=False):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # One-hot encode the target labels
        labels_one_hot = F.one_hot(labels, num_classes=num_classes).float()

        outputs = model(inputs)
        loss = criterion(outputs, labels_one_hot)

        preds = torch.sigmoid(outputs) > 0.5
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels_one_hot.data)
        total_samples += inputs.size(0)

        all_predictions.append(torch.sigmoid(outputs).detach().cpu().numpy())
        all_true_labels.append(labels.cpu().numpy())

    test_loss = running_loss / total_samples
    test_accuracy = running_corrects.double() / total_samples

    # Concatenate all predictions and true labels
    predictions = np.concatenate(all_predictions)
    true_labels = np.concatenate(all_true_labels)

    # Ensure the lengths of true labels and predictions match
    assert true_labels.shape[0] == predictions.shape[0], f"Lengths of true labels ({true_labels.shape[0]}) and predictions ({predictions.shape[0]}) do not match"

    # Flatten the arrays for ROC curve computation
    true_labels_flat = true_labels.ravel()
    predictions_flat = predictions.ravel()

    # Calculate APCER, BPCER, and EER
    apcer_values = []
    bpcer_values = []

    for fixed_bpcer in fixed_bpcer_values:
        apcer = calculate_apcer(true_labels_flat, predictions_flat, fixed_bpcer)
        apcer_values.append(apcer)

    for fixed_apcer in fixed_apcer_values:
        bpcer = calculate_bpcer(true_labels_flat, predictions_flat, fixed_apcer)
        bpcer_values.append(bpcer)

    eer = calculate_eer(true_labels_flat, predictions_flat)

    # Store results
    result = {
        "Dataset": dataset_name,
        "Loss": test_loss,
        "Accuracy": test_accuracy.item(),
        "APCER": apcer_values,
        "BPCER": bpcer_values,
        "EER": eer
    }
    all_results.append(result)

    # Print results
    print(f"{dataset_name} - Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}")
    print(f"{dataset_name} - APCER: {apcer_values}, BPCER: {bpcer_values}, EER: {eer:.4f}")

# Convert the final results to a DataFrame
df_results = pd.DataFrame(all_results)

# Print the final results
print("\nFinal Results:")
print(df_results)


Evaluating dataset: FaceMorpher


ValueError: Found input variables with inconsistent numbers of samples: [1204, 2408]

### **Evaluation Metrics**

In [43]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve
from torch.utils.data import DataLoader

def calculate_apcer(true_labels, predictions, fixed_bpcer):
    """Calculate APCER at a fixed BPCER."""
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    fpr_target = fixed_bpcer
    closest_fpr_index = np.argmin(np.abs(fpr - fpr_target))
    apcer = 1 - tpr[closest_fpr_index]
    return apcer

def calculate_bpcer(true_labels, predictions, fixed_apcer):
    """Calculate BPCER at a fixed APCER."""
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    tpr_target = 1 - fixed_apcer
    closest_tpr_index = np.argmin(np.abs(tpr - tpr_target))
    bpcer = fpr[closest_tpr_index]
    return bpcer

def calculate_eer(true_labels, predictions):
    """Calculate EER."""
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    frr = 1 - tpr
    eer_index = np.argmin(np.abs(fpr - frr))
    eer = fpr[eer_index]
    return eer



In [49]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm import tqdm

def calculate_apcer(true_labels, predictions, fixed_bpcer):
    """Calculate APCER at a fixed BPCER."""
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    fpr_target = fixed_bpcer
    closest_fpr_index = np.argmin(np.abs(fpr - fpr_target))
    apcer = 1 - tpr[closest_fpr_index]
    return apcer

def calculate_bpcer(true_labels, predictions, fixed_apcer):
    """Calculate BPCER at a fixed APCER."""
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    tpr_target = 1 - fixed_apcer
    closest_tpr_index = np.argmin(np.abs(tpr - tpr_target))
    bpcer = fpr[closest_tpr_index]
    return bpcer

def calculate_eer(true_labels, predictions):
    """Calculate EER."""
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    frr = 1 - tpr
    eer_index = np.argmin(np.abs(fpr - frr))
    eer = fpr[eer_index]
    return eer

all_results = []

# Iterate over each dataset
for loader_idx, data_loader in enumerate(data_loaders):
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0
    dataset_name = names[loader_idx]
    print(f"Evaluating dataset: {dataset_name}")

    all_predictions = []
    all_true_labels = []

    for inputs, labels in tqdm(data_loader, desc=f"Processing {dataset_name}", leave=False):
        inputs = inputs.to(device)
        labels = labels.to(device)  # No need to reshape or convert to float

        # One-hot encode the target labels
        labels_one_hot = F.one_hot(labels, num_classes=num_classes).float()

        outputs = model(inputs)
        preds = torch.sigmoid(outputs) > 0.5
        loss = criterion(outputs, labels_one_hot)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels_one_hot.data)
        total_samples += inputs.size(0)

        all_predictions.append(torch.sigmoid(outputs).detach().cpu().numpy())
        all_true_labels.append(labels.cpu().numpy())

    test_loss = running_loss / total_samples
    test_accuracy = running_corrects.double() / total_samples

    # Concatenate all predictions and true labels
    predictions = np.concatenate(all_predictions)
    true_labels = np.concatenate(all_true_labels)

    # Ensure the lengths of true labels and predictions match
    assert true_labels.shape[0] == predictions.shape[0], f"Lengths of true labels ({true_labels.shape[0]}) and predictions ({predictions.shape[0]}) do not match"

    # Flatten the arrays for ROC curve computation
    true_labels_flat = true_labels.ravel()
    predictions_flat = predictions.ravel()

    # Calculate APCER, BPCER, and EER
    apcer_values = []
    bpcer_values = []

    for fixed_bpcer in fixed_bpcer_values:
        apcer = calculate_apcer(true_labels_flat, predictions_flat, fixed_bpcer)
        apcer_values.append(apcer)

    for fixed_apcer in fixed_apcer_values:
        bpcer = calculate_bpcer(true_labels_flat, predictions_flat, fixed_apcer)
        bpcer_values.append(bpcer)

    eer = calculate_eer(true_labels_flat, predictions_flat)

    # Store results
    result = {
        "Dataset": dataset_name,
        "Loss": test_loss,
        "Accuracy": test_accuracy.item(),
        "APCER": apcer_values,
        "BPCER": bpcer_values,
        "EER": eer
    }
    all_results.append(result)

    # Print results
    print(f"{dataset_name} - Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}")
    print(f"{dataset_name} - APCER: {apcer_values}, BPCER: {bpcer_values}, EER: {eer:.4f}")

# Print the final results
print("\nFinal Results:")
for result in all_results:
    print(result)


Evaluating dataset: FaceMorpher


ValueError: Found input variables with inconsistent numbers of samples: [1204, 2408]

In [66]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm import tqdm

def calculate_apcer(true_labels, predictions, fixed_bpcer):
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    closest_fpr_index = np.argmin(np.abs(fpr - fixed_bpcer))
    apcer = 1 - tpr[closest_fpr_index]
    return apcer

def calculate_bpcer(true_labels, predictions, fixed_apcer):
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    closest_tpr_index = np.argmin(np.abs(tpr - (1 - fixed_apcer)))
    bpcer = fpr[closest_tpr_index]
    return bpcer

def calculate_eer(true_labels, predictions):
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    frr = 1 - tpr
    eer_index = np.argmin(np.abs(fpr - frr))
    eer = fpr[eer_index]
    return eer

# Assuming these variables are defined somewhere in your code
# model, criterion, device, num_classes, data_loaders, results, losses, fixed_bpcer_values, fixed_apcer_values

all_results = []

for loader_idx, data_loader in enumerate(data_loaders):
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0
    dataset_name = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"][loader_idx]
    print(f"Evaluating dataset: {dataset_name}")

    all_predictions = []
    all_true_labels = []

    for inputs, labels in tqdm(data_loader, desc=f"Processing {dataset_name}", leave=False):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # One-hot encode the target labels
        labels_one_hot = F.one_hot(labels, num_classes=num_classes).float()

        outputs = model(inputs)
        loss = criterion(outputs, labels_one_hot)

        preds = torch.sigmoid(outputs) > 0.5
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels_one_hot.data)
        total_samples += inputs.size(0)

        all_predictions.append(torch.sigmoid(outputs).detach().cpu().numpy())
        all_true_labels.append(labels.cpu().numpy())

    test_loss = running_loss / total_samples
    test_accuracy = running_corrects.double() / total_samples

    # Concatenate all predictions and true labels
    predictions = np.concatenate(all_predictions)
    true_labels = np.concatenate(all_true_labels)

    # Ensure the lengths of true labels and predictions match
    assert true_labels.shape[0] == predictions.shape[0], f"Lengths of true labels ({true_labels.shape[0]}) and predictions ({predictions.shape[0]}) do not match"

    # Flatten the arrays for ROC curve computation
    true_labels_flat = true_labels.ravel()
    predictions_flat = predictions.ravel()

    # Calculate APCER, BPCER, and EER separately
    apcer_values = [calculate_apcer(true_labels_flat, predictions_flat, fixed_bpcer) for fixed_bpcer in fixed_bpcer_values]
    bpcer_values = [calculate_bpcer(true_labels_flat, predictions_flat, fixed_apcer) for fixed_apcer in fixed_apcer_values]
    eer = calculate_eer(true_labels_flat, predictions_flat)

    # Store results
    result = {
        "Dataset": dataset_name,
        "Loss": test_loss,
        "Accuracy": test_accuracy.item(),
        "APCER": apcer_values,
        "BPCER": bpcer_values,
        "EER": eer
    }
    all_results.append(result)

    # Print results
    print(f"{dataset_name} - Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}")
    print(f"{dataset_name} - APCER: {apcer_values}, BPCER: {bpcer_values}, EER: {eer:.4f}")

# Convert the final results to a DataFrame
df_results = pd.DataFrame(all_results)

# Print the final results
print("\nFinal Results:")
print(df_results)


Evaluating dataset: FaceMorpher


ValueError: Found input variables with inconsistent numbers of samples: [1204, 2408]